In [2]:
from diff_predictor import data_process, predxgboost, spatial
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 

from os import listdir, getcwd, chdir
from os.path import isfile, join
import os
from sklearn.preprocessing import scale, StandardScaler
from numpy.random import permutation


from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, recall_score, precision_score, f1_score
import operator
import xgboost as xgb
import shap
from xgboost.training import CVPack
from xgboost import callback
from xgboost.core import CallbackEnv
from xgboost.core import EarlyStopException
from xgboost.core import STRING_TYPES

import mlflow
import warnings

Using current directory for loading/saving: /Users/nelsschimek/Documents/nancelab/diff_predictor/notebooks
To change current directory, call diff_predictor.core.change_dir(...)


In [3]:
workbookDir = getcwd()

print('Current Notebook Dir: ' + workbookDir)
chdir(workbookDir) # Go to current workbook Dir"
chdir('..')        # Go up one
print(f'Using current directory for loading data: {getcwd()}')
workbookDir = getcwd()

Current Notebook Dir: /Users/nelsschimek/Documents/nancelab/diff_predictor/notebooks
Using current directory for loading data: /Users/nelsschimek/Documents/nancelab/diff_predictor


In [4]:
rotenone_feature_path = workbookDir + '/data/Brendan_traj_data/Cortex_features/'
rotenone_feature_filelist = [f for f in listdir(rotenone_feature_path) if isfile(join(rotenone_feature_path, f)) and 'feat' in f and '7DIV' in f ]
print(len(rotenone_feature_filelist))
print(rotenone_feature_filelist)

8
['features_P10F_5uM_7DIV_40nm_slice_1_cortex_vid_1.csv', 'features_P10F_NT_7DIV_40nm_slice_1_cortex_vid_1.csv', 'features_P10F_NT_7DIV_40nm_slice_1_cortex_vid_2.csv', 'features_P10F_1uM_7DIV_40nm_slice_1_cortex_vid_1.csv', 'features_P10F_50nM_7DIV_40nm_slice_1_cortex_vid_2.csv', 'features_P10F_5uM_7DIV_40nm_slice_2_cortex_vid_2.csv', 'features_P10F_1uM_7DIV_40nm_slice_1_cortex_vid_2.csv', 'features_P10F_50nM_7DIV_40nm_slice_1_cortex_vid_1.csv']


In [5]:
fstats_tot_rotenone = data_process.generate_fullstats(rotenone_feature_path, rotenone_feature_filelist, ['NT','50nM', '1uM', '5uM'], 'dosage')

Adding file features_P10F_5uM_7DIV_40nm_slice_1_cortex_vid_1.csv size: (2405, 68)
Adding file features_P10F_NT_7DIV_40nm_slice_1_cortex_vid_1.csv size: (1738, 68)
Adding file features_P10F_NT_7DIV_40nm_slice_1_cortex_vid_2.csv size: (1190, 68)
Adding file features_P10F_1uM_7DIV_40nm_slice_1_cortex_vid_1.csv size: (638, 68)
Adding file features_P10F_50nM_7DIV_40nm_slice_1_cortex_vid_2.csv size: (1785, 68)
Adding file features_P10F_5uM_7DIV_40nm_slice_2_cortex_vid_2.csv size: (2829, 68)
Adding file features_P10F_1uM_7DIV_40nm_slice_1_cortex_vid_2.csv size: (863, 68)
Adding file features_P10F_50nM_7DIV_40nm_slice_1_cortex_vid_1.csv size: (2632, 68)


In [6]:
feature_list = [
    'alpha', # Fitted anomalous diffusion alpha exponenet
    'D_fit', # Fitted anomalous diffusion coefficient
    'kurtosis', # Kurtosis of track
    'asymmetry1', # Asymmetry of trajecory (0 for circular symmetric, 1 for linear)
    'asymmetry2', # Ratio of the smaller to larger principal radius of gyration
    'asymmetry3', # An asymmetric feature that accnts for non-cylindrically symmetric pt distributions
    'AR', # Aspect ratio of long and short side of trajectory's minimum bounding rectangle
    'elongation', # Est. of amount of extension of trajectory from centroid
    'boundedness', # How much a particle with Deff is restricted by a circular confinement of radius r
    'fractal_dim', # Measure of how complicated a self similar figure is
    'trappedness', # Probability that a particle with Deff is trapped in a region
    'efficiency', # Ratio of squared net displacement to the sum of squared step lengths
    'straightness', # Ratio of net displacement to the sum of squared step lengths
    'MSD_ratio', # MSD ratio of the track
#     'frames', # Number of frames the track spans
    'Deff1', # Effective diffusion coefficient at 0.33 s
    'Deff2', # Effective diffusion coefficient at 3.3 s
    #'angle_mean', # Mean turning angle which is counterclockwise angle from one frame point to another
    #'angle_mag_mean', # Magnitude of the turning angle mean
    #'angle_var', # Variance of the turning angle
    #'dist_tot', # Total distance of the trajectory
    #'dist_net', # Net distance from first point to last point
    #'progression', # Ratio of the net distance traveled and the total distance
    'Mean alpha', 
    'Mean D_fit', 
    'Mean kurtosis', 
    'Mean asymmetry1', 
    'Mean asymmetry2',
    'Mean asymmetry3', 
    'Mean AR',
    'Mean elongation', 
    'Mean boundedness',
    'Mean fractal_dim', 
    'Mean trappedness', 
    'Mean efficiency',
    'Mean straightness', 
    'Mean MSD_ratio', 
    'Mean Deff1', 
    'Mean Deff2',
    ]

target = 'dosage'

In [81]:
  
mean_feature_list = [ 
   'Mean alpha', 
    'Mean D_fit', 
    'Mean kurtosis', 
    'Mean asymmetry1', 
    'Mean asymmetry2',
    'Mean asymmetry3', 
    'Mean AR',
    'Mean elongation', 
    'Mean boundedness',
    'Mean fractal_dim', 
    'Mean trappedness', 
    'Mean efficiency',
    'Mean straightness', 
    'Mean MSD_ratio', 
    'Mean Deff1', 
    'Mean Deff2',
    ]

In [95]:
ecm = fstats_tot_rotenone[feature_list + [target, 'Track_ID', 'X', 'Y']] #dont think i need these rn
print(ecm.shape)
ecm = ecm[~ecm[list(set(feature_list) - set(['Deff2', 'Mean Deff2']))].isin([np.nan, np.inf, -np.inf]).any(1)]       # Removing nan and inf data points
ecm.shape

(14080, 36)


(12493, 36)

In [96]:
bal_ecm = data_process.balance_data(ecm, target)
bal_ecm = bal_ecm.reset_index(drop=True)
sampled_df = data_process.bin_data(bal_ecm)

Ratio before data balance (5uM:NT:1uM:50nM) = 4577:2759:1289:3868
Ratio after balance (5uM:NT:1uM:50nM) = 1289:1289:1289:1289


In [104]:
le = preprocessing.LabelEncoder()
sampled_df['encoded_target'] = le.fit_transform(sampled_df[target])

seed = 1234
np.random.seed(seed)
train_split = 0.8
test_split = 0.5


training_bins = np.random.choice(sampled_df['bins'].unique(), int(len(sampled_df['bins'].unique())*train_split), replace=False)

X_train = sampled_df[sampled_df['bins'].isin(training_bins)]
X_test_val = sampled_df[~sampled_df['bins'].isin(training_bins)]
X_val, X_test = train_test_split(X_test_val, test_size=test_split, random_state=seed)

y_train = X_train['encoded_target']
y_test = X_test['encoded_target']
y_val = X_val['encoded_target']

dtrain = xgb.DMatrix(X_train[feature_list], label=y_train)
dtest = xgb.DMatrix(X_test[feature_list], label=y_test)
dval = xgb.DMatrix(X_val[feature_list], label=y_val)

model = xgb.XGBClassifier().fit(X_train[feature_list], y_train)
model.score(X_train[feature_list], y_train)

0.8832134292565947

In [107]:
model.score(X_val[feature_list], y_val)

0.5070993914807302

In [99]:
booster, acc, true_label, preds = predxgboost.train(None, dtrain, dtest, dval, evals=[(dtrain, 'train'), (dval, 'eval')], num_round=200, verbose=False)


TypeError: argument of type 'NoneType' is not iterable

In [11]:
from feature_engine import selection

In [91]:
param = {'max_depth': 3,
         'eta': 0.005,
         'min_child_weight': 0,
         'verbosity': 0,
         'objective': 'multi:softprob',
         'num_class': 4,
         'silent': 'True',
         'gamma': 5,
         'subsample': 0.15,
         'colsample_bytree': 0.8,
         'eval_metric': "mlogloss",
#          # GPU integration will cut time in ~half:
#          'gpu_id' : 0,
#          'tree_method': 'gpu_hist',
#          'predictor': 'gpu_predictor'
        }

estimator = xgb.XGBClassifier(max_depth=3,
         eta= 0.005,
         min_child_weight= 0,
         verbosity= 0,
         objective= 'multi:softprob',
         num_class=4,
         silent=True,
         gamma=5,
         subsample= 0.15,
         colsample_bytree= 0.8,
         eval_metric= "mlogloss")

In [92]:
selector = selection.RecursiveFeatureAddition(estimator=estimator, variables=feature_list, scoring='accuracy')

In [93]:
selector.fit(X_train, y_train)

RecursiveFeatureAddition(estimator=XGBClassifier(colsample_bytree=0.8,
                                                 eta=0.005,
                                                 eval_metric='mlogloss',
                                                 gamma=5, min_child_weight=0,
                                                 num_class=4,
                                                 objective='multi:softprob',
                                                 silent=True, subsample=0.15,
                                                 verbosity=0),
                         scoring='accuracy',
                         variables=['alpha', 'D_fit', 'kurtosis', 'asymmetry1',
                                    'asymmetry2', 'asymmetry3', 'AR',
                                    'elongation', 'boundedness', 'fractal_dim',
                                    'trappedness', 'efficiency', 'straightness',
                                    'MSD_ratio', 'Deff1', 'Deff2', 'Mean alpha',
 

In [94]:
selector.initial_model_performance_

0.692326139088729

In [74]:
reduced_ecm = reduced_ecm.
bal_ecm = data_process.balance_data(reduced_ecm, target)
bal_ecm = bal_ecm.reset_index(drop=True)
sampled_df = data_process.bin_data(bal_ecm)
sampled_df

Ratio before data balance (5uM:NT:1uM:50nM) = 4577:2759:1289:3868
Ratio after balance (5uM:NT:1uM:50nM) = 1289:1289:1289:1289


,Mean asymmetry2,Mean asymmetry3,Mean fractal_dim,Mean efficiency,Mean straightness,Mean Deff1,dosage,X,Y,binx,biny,bins
0,0.203237,0.301646,1.662087,1.079506,0.279614,8.590620,5uM,1037.696041,1351.996558,8,10,138
1,0.137609,0.390748,1.934988,0.663315,0.239634,0.679439,5uM,1191.165927,915.868137,9,7,151
2,0.270627,0.240342,1.573744,1.143421,0.286556,11.339848,5uM,203.846022,795.735566,1,6,22
3,0.232761,0.324803,1.787140,0.804416,0.156205,1.967246,5uM,2026.467914,962.510726,15,7,247
4,0.258161,0.239906,1.659454,0.833392,0.245221,3.773468,5uM,1166.918567,971.703498,9,7,151
...,...,...,...,...,...,...,...,...,...,...,...,...
5151,0.172426,0.391897,2.118817,0.239830,0.106523,0.628064,50nM,243.702244,1243.773987,1,9,25
5152,0.202545,0.332361,1.729918,0.863326,0.240339,2.188152,50nM,1493.884639,881.514470,11,6,182
5153,0.318182,0.224856,1.638479,1.060995,0.282377,4.421858,50nM,1390.299500,967.031000,10,7,167
5154,0.301938,0.203381,2.015782,0.373216,0.124503,0.422372,50nM,1081.075140,571.115081,8,4,132


In [75]:
reduced_feature_list = ['Mean asymmetry2', 'Mean asymmetry3', 'Mean fractal_dim', 'Mean efficiency', 'Mean straightness', 'Mean Deff1']

In [85]:
le = preprocessing.LabelEncoder()
sampled_df['encoded_target'] = le.fit_transform(sampled_df[target])

seed = 1234
np.random.seed(seed)
train_split = 0.8
test_split = 0.5


training_bins = np.random.choice(sampled_df['bins'].unique(), int(len(sampled_df['bins'].unique())*train_split), replace=False)

X_train = sampled_df[sampled_df['bins'].isin(training_bins)]
X_test_val = sampled_df[~sampled_df['bins'].isin(training_bins)]
X_val, X_test = train_test_split(X_test_val, test_size=test_split, random_state=seed)

y_train = X_train['encoded_target']
y_test = X_test['encoded_target']
y_val = X_val['encoded_target']

dtrain = xgb.DMatrix(X_train[mean_feature_list], label=y_train)
dtest = xgb.DMatrix(X_test[mean_feature_list], label=y_test)
dval = xgb.DMatrix(X_val[mean_feature_list], label=y_val)

In [87]:
booster, acc, true_label, preds = predxgboost.train(param, dtrain, dtest, dval, evals=[(dtrain, 'train'), (dval, 'eval')], num_round=200, verbose=True)


[0]	train-mlogloss:1.3839	eval-mlogloss:1.38439
[1]	train-mlogloss:1.38172	eval-mlogloss:1.38256
[2]	train-mlogloss:1.3795	eval-mlogloss:1.3811
[3]	train-mlogloss:1.3774	eval-mlogloss:1.3796
[4]	train-mlogloss:1.37515	eval-mlogloss:1.37791
[5]	train-mlogloss:1.37286	eval-mlogloss:1.37588
[6]	train-mlogloss:1.37055	eval-mlogloss:1.37416
[7]	train-mlogloss:1.36842	eval-mlogloss:1.37267
[8]	train-mlogloss:1.36609	eval-mlogloss:1.37074
[9]	train-mlogloss:1.36375	eval-mlogloss:1.36872
[10]	train-mlogloss:1.36158	eval-mlogloss:1.36715
[11]	train-mlogloss:1.35935	eval-mlogloss:1.3651
[12]	train-mlogloss:1.35705	eval-mlogloss:1.36334
[13]	train-mlogloss:1.35493	eval-mlogloss:1.36192
[14]	train-mlogloss:1.35281	eval-mlogloss:1.36021
[15]	train-mlogloss:1.35052	eval-mlogloss:1.35874
[16]	train-mlogloss:1.34833	eval-mlogloss:1.35701
[17]	train-mlogloss:1.34612	eval-mlogloss:1.35563
[18]	train-mlogloss:1.34381	eval-mlogloss:1.35383
[19]	train-mlogloss:1.34173	eval-mlogloss:1.35245
[20]	train-mlogl